In [2]:
from ts_vae.data_processors.ts_gen_processor import TSGenDataset
from torch_geometric.data import DataLoader
import numpy as np
#from ts_vae.utils import remove_files
#remove_files() 
rxns = TSGenDataset(r'data')

In [3]:
tt_split = 0.8
num_rxns = len(rxns)
num_train = int(np.floor(tt_split * num_rxns))
batch_size = 4
#to_follow = ['edge_index_r', 'edge_index_ts', 'edge_index_p', 'edge_attr_r', 'edge_attr_ts', 'edge_attr_p'
#                'pos_r', 'pos_ts', 'pos_p', 'x_r', 'x_ts', 'x_p']
train_loader = DataLoader(rxns[: num_train], batch_size = batch_size)
test_loader = DataLoader(rxns[num_train: ], batch_size = batch_size)

In [48]:
# rxns[0]
batch = next(iter(train_loader))
test = batch.edge_attr

In [55]:
len(batch.idx)

4

In [49]:
b1 = batch[0].edge_attr.view(441, 3)
t1 = test.view(4, 441, 3)
(b1 == t1[0]).all()

tensor(True)

torch.Size([21, 21, 3])

In [8]:
def collate_fn(batch):
    # batch: list of datapoints to be collated
    # return dict of pytorch tensors
    

    batch = {prop: batch_stack([mol[prop] for mol in batch]) for prop in batch[0].keys()}

    to_keep = (batch['charges'].sum(0) > 0)

    batch = {key: drop_zeros(prop, to_keep) for key, prop in batch.items()}

    atom_mask = batch['charges'] > 0
    batch['atom_mask'] = atom_mask

    #Obtain edges
    batch_size, n_nodes = atom_mask.size()
    edge_mask = atom_mask.unsqueeze(1) * atom_mask.unsqueeze(2)

    #mask diagonal
    diag_mask = ~torch.eye(edge_mask.size(1), dtype=torch.bool).unsqueeze(0)
    edge_mask *= diag_mask

    #edge_mask = atom_mask.unsqueeze(1) * atom_mask.unsqueeze(2)
    batch['edge_mask'] = edge_mask.view(batch_size * n_nodes * n_nodes, 1)


    return batch

In [6]:
import torch
a = torch.randn(2, 21, 21, 100)
torch.sum(a, (2, 0)).shape

torch.Size([21, 100])

In [1]:
from experiments.building_on_mit.meta_eval.meta_eval import ablation_experiment
# from ts_vae.utils import remove_files
# remove_files()
# have to use batch_size = 1 right now
train_log, test_log = ablation_experiment(0.8, 2, 2, 2)

Starting ablation experiment...
gnn edge out torch.Size([882, 100])
edge out post mlp 1 torch.Size([882, 100])
emb torch.Size([2, 100])
edge out post mlp 2 torch.Size([2, 21, 21, 2])
edge out post tranpose torch.Size([2, 21, 21, 2])
dw pred torch.Size([2, 21, 21, 2])
D init vec torch.Size([2, 21, 21])
W vec torch.Size([2, 21, 21])
torch.Size([2, 21, 21])


RuntimeError: shape '[1, 1]' is invalid for input of size 882

In [19]:
# dir(train_log)
train_log.epoch_ae_results[0]['D_init'][0].shape

torch.Size([21, 21])

- How to batch edge_attr (dim = NxNxEA) properly? i.e. on NxN
- Sort the collate for the tsgendatset. What is diff between collate for loader and inmemorydataset?
    - Loader: pass in Collater class with collate() for batching
        - The main issue is in Collater(), you need to write a Batch.from_data_list() func which is q long.
    - InMemoryDataset: collate is for slicing appropriately. You'd then need to specify follow_batch etc
<br><br>
- Ignore above, just get working for constant size and batch length.
- Try and replicate their results. Main thing to do.